# Devices and Virtual Devices

In [1]:
import numpy as np
from pulser.devices import Device, VirtualDevice, Chadoq2, MockDevice
from pulser.channels import Rydberg, Raman, Microwave
from pulser import Pulse, Sequence, Register

## The `Device` class

### Physical devices: example of Chadoq2

Some physical devices are accessible on `pulser.devices`. These devices are instances of the `Device` class. Instances of this class are constrained by physical considerations, and all its properties have to be defined for it to be valid. Below is presented the specifications of the physical device `Chadoq2`, that are accessible via the `print_specs` method. 

In [2]:
# Dipslay Chadoq2's specifications
Chadoq2.print_specs()

----------------------
Chadoq2 Specifications
----------------------

Register parameters:
 - Dimensions: 2D
 - Rydberg level: 70
 - Maximum number of atoms: 100
 - Maximum distance from origin: 50 μm
 - Minimum distance between neighbouring atoms: 4 μm
 - Maximum layout filling fraction: 0.5
 - SLM Mask: Yes

Channels:
 - 'rydberg_global': Rydberg.Global(Max Absolute Detuning: 125.66370614359172 rad/µs, Max Amplitude: 15.707963267948966 rad/µs, Clock period: 4 ns, Minimum pulse duration: 16 ns, Maximum pulse duration: 67108864 ns, Basis: 'ground-rydberg')
 - 'rydberg_local': Rydberg.Local(Max Absolute Detuning: 125.66370614359172 rad/µs, Max Amplitude: 62.83185307179586 rad/µs, Minimum retarget time: 220 ns, Fixed retarget time: 0 ns, Max targets: 1, Clock period: 4 ns, Minimum pulse duration: 16 ns, Maximum pulse duration: 67108864 ns, Basis: 'ground-rydberg')
 - 'raman_local': Raman.Local(Max Absolute Detuning: 125.66370614359172 rad/µs, Max Amplitude: 62.83185307179586 rad/µs, Mini

### Defining your own physical device

It is possible to define your own device using the `Device` class. For example, it could be of interest, as a mind game, to use a QPU having the same properties as `Chadoq2`, but with a higher dimension.

In [3]:
Chadoq3D = Device(
    name="Chadoq3D",
    dimensions=3,  # dimensions=2 on Chadoq2
    rydberg_level=70,
    max_atom_num=100,
    max_radial_distance=50,
    min_atom_distance=4,
    supports_slm_mask=True,
    channel_objects=(
        Rydberg.Global(
            max_abs_detuning=2 * np.pi * 20,
            max_amp=2 * np.pi * 2.5,
            clock_period=4,
            min_duration=16,
            max_duration=2**26,
        ),
        Rydberg.Local(
            max_abs_detuning=2 * np.pi * 20,
            max_amp=2 * np.pi * 10,
            min_retarget_interval=220,
            fixed_retarget_t=0,
            max_targets=1,
            clock_period=4,
            min_duration=16,
            max_duration=2**26,
        ),
        Raman.Local(
            max_abs_detuning=2 * np.pi * 20,
            max_amp=2 * np.pi * 10,
            min_retarget_interval=220,
            fixed_retarget_t=0,
            max_targets=1,
            clock_period=4,
            min_duration=16,
            max_duration=2**26,
        ),
    ),
)
Chadoq3D.print_specs()

-----------------------
Chadoq3D Specifications
-----------------------

Register parameters:
 - Dimensions: 3D
 - Rydberg level: 70
 - Maximum number of atoms: 100
 - Maximum distance from origin: 50 μm
 - Minimum distance between neighbouring atoms: 4 μm
 - Maximum layout filling fraction: 0.5
 - SLM Mask: Yes

Channels:
 - 'rydberg_global': Rydberg.Global(Max Absolute Detuning: 125.66370614359172 rad/µs, Max Amplitude: 15.707963267948966 rad/µs, Clock period: 4 ns, Minimum pulse duration: 16 ns, Maximum pulse duration: 67108864 ns, Basis: 'ground-rydberg')
 - 'rydberg_local': Rydberg.Local(Max Absolute Detuning: 125.66370614359172 rad/µs, Max Amplitude: 62.83185307179586 rad/µs, Minimum retarget time: 220 ns, Fixed retarget time: 0 ns, Max targets: 1, Clock period: 4 ns, Minimum pulse duration: 16 ns, Maximum pulse duration: 67108864 ns, Basis: 'ground-rydberg')
 - 'raman_local': Raman.Local(Max Absolute Detuning: 125.66370614359172 rad/µs, Max Amplitude: 62.83185307179586 rad/µs, M

It can also be relevant to add channels, such as `Microwave` that encodes the "XY" basis. The channels enabled on the device are defined in `channel_objects`. Their name can be defined in `channel_id`. If no name is provided, the name of the channels are automatically generated as `channeltype_adressing`. For example, the `Rydberg.Global` channel is automatically named `rydberg_global` in the specifications of `Chadoq2`.

In [4]:
Chadoq3DXY = Device(
    name="Chadoq3DXY",
    dimensions=3,  # dimensions=2 on Chadoq2
    rydberg_level=70,
    max_atom_num=100,
    max_radial_distance=50,
    min_atom_distance=4,
    interaction_coeff_xy=3700.0,
    supports_slm_mask=True,
    channel_ids=("ryd_glob", "ryd_loc", "ram_loc", "mw_glob"),
    channel_objects=(
        Rydberg.Global(
            max_abs_detuning=2 * np.pi * 20,
            max_amp=2 * np.pi * 2.5,
            clock_period=4,
            min_duration=16,
            max_duration=2**26,
        ),
        Rydberg.Local(
            max_abs_detuning=2 * np.pi * 20,
            max_amp=2 * np.pi * 10,
            min_retarget_interval=220,
            fixed_retarget_t=0,
            max_targets=1,
            clock_period=4,
            min_duration=16,
            max_duration=2**26,
        ),
        Raman.Local(
            max_abs_detuning=2 * np.pi * 20,
            max_amp=2 * np.pi * 10,
            min_retarget_interval=220,
            fixed_retarget_t=0,
            max_targets=1,
            clock_period=4,
            min_duration=16,
            max_duration=2**26,
        ),
        Microwave.Global(
            max_abs_detuning=2 * np.pi * 20,
            max_amp=2 * np.pi * 2.5,
            clock_period=4,
            min_duration=16,
            max_duration=2**26,
        ),
    ),
)
Chadoq3DXY.print_specs()

-------------------------
Chadoq3DXY Specifications
-------------------------

Register parameters:
 - Dimensions: 3D
 - Rydberg level: 70
 - Maximum number of atoms: 100
 - Maximum distance from origin: 50 μm
 - Minimum distance between neighbouring atoms: 4 μm
 - Maximum layout filling fraction: 0.5
 - SLM Mask: Yes

Channels:
 - 'ryd_glob': Rydberg.Global(Max Absolute Detuning: 125.66370614359172 rad/µs, Max Amplitude: 15.707963267948966 rad/µs, Clock period: 4 ns, Minimum pulse duration: 16 ns, Maximum pulse duration: 67108864 ns, Basis: 'ground-rydberg')
 - 'ryd_loc': Rydberg.Local(Max Absolute Detuning: 125.66370614359172 rad/µs, Max Amplitude: 62.83185307179586 rad/µs, Minimum retarget time: 220 ns, Fixed retarget time: 0 ns, Max targets: 1, Clock period: 4 ns, Minimum pulse duration: 16 ns, Maximum pulse duration: 67108864 ns, Basis: 'ground-rydberg')
 - 'ram_loc': Raman.Local(Max Absolute Detuning: 125.66370614359172 rad/µs, Max Amplitude: 62.83185307179586 rad/µs, Minimum ret

Of course, the other parameters can be modified. However, all these parameters have to be defined and you cannot change the Rydberg level used in the device after having declared it. A more flexible way of defining a device on an emulator is to define an instance of the `VirtualDevice` class.

## The `VirtualDevice` class

### Converting a Device in a Virtual Device

The `VirtualDevice` class is useful to define a device to run on an emulator. As the device built in the previous cell `Chadoq3DXY` does not exist physically, it is better to create a virtual device to perform experiment with it. This is possible by using the `to_virtual` method.

In [5]:
# Converting the Device object in a VirtualDevice object
VirtualChadoq3DXY = Chadoq3DXY.to_virtual(reusable_channels_to_virtual=True)
print(VirtualChadoq3DXY)

VirtualDevice(name='Chadoq3DXY', dimensions=3, rydberg_level=70, min_atom_distance=4, max_atom_num=100, max_radial_distance=50, interaction_coeff_xy=3700.0, supports_slm_mask=True, max_layout_filling=0.5, reusable_channels=True, channel_ids=('ryd_glob', 'ryd_loc', 'ram_loc', 'mw_glob'), channel_objects=(Rydberg.Global(Max Absolute Detuning: 125.66370614359172 rad/µs, Max Amplitude: 15.707963267948966 rad/µs, Clock period: 4 ns, Minimum pulse duration: 16 ns, Maximum pulse duration: 67108864 ns, Basis: 'ground-rydberg'), Rydberg.Local(Max Absolute Detuning: 125.66370614359172 rad/µs, Max Amplitude: 62.83185307179586 rad/µs, Minimum retarget time: 220 ns, Fixed retarget time: 0 ns, Max targets: 1, Clock period: 4 ns, Minimum pulse duration: 16 ns, Maximum pulse duration: 67108864 ns, Basis: 'ground-rydberg'), Raman.Local(Max Absolute Detuning: 125.66370614359172 rad/µs, Max Amplitude: 62.83185307179586 rad/µs, Minimum retarget time: 220 ns, Fixed retarget time: 0 ns, Max targets: 1, Cloc

It is possible to manipulate the virtual device in a way that would not have been possible physically. These manipulations are the possible change of the Rydberg level used in the experiment and the possibility to call a channel multiple times in the same sequence.

In [6]:
# Changing the Rydberg level
VirtualChadoq3DXY.change_rydberg_level(60)
print(VirtualChadoq3DXY)

VirtualDevice(name='Chadoq3DXY', dimensions=3, rydberg_level=60, min_atom_distance=4, max_atom_num=100, max_radial_distance=50, interaction_coeff_xy=3700.0, supports_slm_mask=True, max_layout_filling=0.5, reusable_channels=True, channel_ids=('ryd_glob', 'ryd_loc', 'ram_loc', 'mw_glob'), channel_objects=(Rydberg.Global(Max Absolute Detuning: 125.66370614359172 rad/µs, Max Amplitude: 15.707963267948966 rad/µs, Clock period: 4 ns, Minimum pulse duration: 16 ns, Maximum pulse duration: 67108864 ns, Basis: 'ground-rydberg'), Rydberg.Local(Max Absolute Detuning: 125.66370614359172 rad/µs, Max Amplitude: 62.83185307179586 rad/µs, Minimum retarget time: 220 ns, Fixed retarget time: 0 ns, Max targets: 1, Clock period: 4 ns, Minimum pulse duration: 16 ns, Maximum pulse duration: 67108864 ns, Basis: 'ground-rydberg'), Raman.Local(Max Absolute Detuning: 125.66370614359172 rad/µs, Max Amplitude: 62.83185307179586 rad/µs, Minimum retarget time: 220 ns, Fixed retarget time: 0 ns, Max targets: 1, Cloc

In [7]:
# Creating a square register
reg = Register.square(4, spacing=5)  # 4x4 array with atoms 5 um apart
# Building a sequence with the register and the virtual device
seq = Sequence(reg, VirtualChadoq3DXY)
# Declare twice the channel "ram_loc"
seq.declare_channel("ch0", "ram_loc")
seq.declare_channel("ch1", "ram_loc")
# Show the declared channels
print(seq.declared_channels)

{'ch0': Raman.Local(Max Absolute Detuning: 125.66370614359172 rad/µs, Max Amplitude: 62.83185307179586 rad/µs, Minimum retarget time: 220 ns, Fixed retarget time: 0 ns, Max targets: 1, Clock period: 4 ns, Minimum pulse duration: 16 ns, Maximum pulse duration: 67108864 ns, Basis: 'digital'), 'ch1': Raman.Local(Max Absolute Detuning: 125.66370614359172 rad/µs, Max Amplitude: 62.83185307179586 rad/µs, Minimum retarget time: 220 ns, Fixed retarget time: 0 ns, Max targets: 1, Clock period: 4 ns, Minimum pulse duration: 16 ns, Maximum pulse duration: 67108864 ns, Basis: 'digital')}


### Building your own virtual device

The `VirtualDevice` class is on the whole more permissive than the `Device` class. In fact, some parameters can be left undefined and easier devices can be built by creating a `VirtualDevice` instance. 

In [8]:
# This basic device can be used for digital quantum computing
BasicMockDevice = VirtualDevice(
    name="BasicMockDevice",
    dimensions=2,
    rydberg_level=70,
    channel_objects=(
        Rydberg.Local(None, None, max_duration=None),
        Raman.Local(None, None, max_duration=None),
    ),
)
print(BasicMockDevice)

VirtualDevice(name='BasicMockDevice', dimensions=2, rydberg_level=70, min_atom_distance=0, max_atom_num=None, max_radial_distance=None, interaction_coeff_xy=None, supports_slm_mask=True, max_layout_filling=0.5, reusable_channels=True, channel_ids=('rydberg_local', 'raman_local'), channel_objects=(Rydberg.Local(Max Absolute Detuning: None, Max Amplitude: None, Minimum retarget time: 0 ns, Fixed retarget time: 0 ns, Clock period: 1 ns, Minimum pulse duration: 1 ns, Basis: 'ground-rydberg'), Raman.Local(Max Absolute Detuning: None, Max Amplitude: None, Minimum retarget time: 0 ns, Fixed retarget time: 0 ns, Clock period: 1 ns, Minimum pulse duration: 1 ns, Basis: 'digital')), _channels=())


Actually, there exists a virtual device with all functionalities, the `MockDevice`.

In [9]:
MyMockDevice = MockDevice
print(MyMockDevice)

VirtualDevice(name='MockDevice', dimensions=3, rydberg_level=70, min_atom_distance=0.0, max_atom_num=None, max_radial_distance=None, interaction_coeff_xy=3700.0, supports_slm_mask=True, max_layout_filling=0.5, reusable_channels=True, channel_ids=('rydberg_global', 'rydberg_local', 'raman_global', 'raman_local', 'mw_global'), channel_objects=(Rydberg.Global(Max Absolute Detuning: None, Max Amplitude: None, Clock period: 1 ns, Minimum pulse duration: 1 ns, Basis: 'ground-rydberg'), Rydberg.Local(Max Absolute Detuning: None, Max Amplitude: None, Minimum retarget time: 0 ns, Fixed retarget time: 0 ns, Clock period: 1 ns, Minimum pulse duration: 1 ns, Basis: 'ground-rydberg'), Raman.Global(Max Absolute Detuning: None, Max Amplitude: None, Clock period: 1 ns, Minimum pulse duration: 1 ns, Basis: 'digital'), Raman.Local(Max Absolute Detuning: None, Max Amplitude: None, Minimum retarget time: 0 ns, Fixed retarget time: 0 ns, Clock period: 1 ns, Minimum pulse duration: 1 ns, Basis: 'digital'), 